In [1]:
# https://github.com/priya-dwivedi/Deep-Learning/blob/master/Object_Detection_Tensorflow_API.ipynb
# https://github.com/priya-dwivedi/Deep-Learning/blob/master/Mask_RCNN/Mask_RCNN_Videos.ipynb

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
print(tf.__version__)

from utils import label_map_util

from utils import visualization_utils as vis_util

1.8.0


utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/local/Cellar/python@2/2.7.14_3/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/Cellar/python@2/2.7.14_3/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/victor/virtualenvlist/mydlp2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/victor/virtualenvlist/mydlp2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start

In [2]:

# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90


In [3]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [4]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [5]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [6]:
# Import moviepy module, you'll need to edit/save/watch video clips later on
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
def detect_videos(image_np, sess, detection_graph):
    
    with detection_graph.as_default():
        
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes', 'detection_masks'
          ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
        if 'detection_masks' in tensor_dict:
            # The following processing is only for single image
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
            detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
            # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
            real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
            detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
            detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
            detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                detection_masks, detection_boxes, image_np.shape[0], image_np.shape[1])
            detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)
            # Follow the convention by adding back the batch dimension
            tensor_dict['detection_masks'] = tf.expand_dims(
                detection_masks_reframed, 0)
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
        output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image_np, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
        if 'detection_masks' in output_dict:
            output_dict['detection_masks'] = output_dict['detection_masks'][0]

        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=1)
                
    return image_np

In [8]:
def process_image(image):  
    global counter
    if counter%1 ==0:
        with detection_graph.as_default():
            with tf.Session(graph=detection_graph) as sess:
                image_np = detect_videos(image, sess, detection_graph) 
    counter +=1 
    return image

In [46]:
input_filename = 'test_images/video1-1.mp4'
output_filename = 'test_images/video1-2.mp4'

counter = 0


clip_playing = VideoFileClip(input_filename)
print("clip.size={}".format(clip_playing.size))
clip_playing.subclip(0, clip_playing.duration)  # suclip(t_start, t_end)

# NOTE: this function expects color images!!s
white_clip = clip_playing.fl_image(process_image) 

# write video to path with progress 
%time white_clip.write_videofile(output_filename)

clip.size=[1080, 1920]
[MoviePy] >>>> Building video test_images/video1-2.mp4
[MoviePy] Writing audio in video1-2TEMP_MPY_wvf_snd.mp3





  0%|          | 0/375 [00:00<?, ?it/s]


 15%|█▍        | 55/375 [00:00<00:00, 529.95it/s]


 31%|███       | 115/375 [00:00<00:00, 556.86it/s]


 48%|████▊     | 179/375 [00:00<00:00, 581.77it/s]


 65%|██████▌   | 244/375 [00:00<00:00, 596.03it/s]


 81%|████████▏ | 305/375 [00:00<00:00, 595.81it/s]


100%|█████████▉| 374/375 [00:00<00:00, 609.35it/s]


100%|██████████| 375/375 [00:00<00:00, 603.78it/s]

[MoviePy] Done.
[MoviePy] Writing video test_images/video1-2.mp4





  0%|          | 0/510 [00:00<?, ?it/s]


  0%|          | 1/510 [00:02<24:53,  2.93s/it]


  0%|          | 2/510 [00:06<26:29,  3.13s/it]


  1%|          | 3/510 [00:09<26:40,  3.16s/it]


  1%|          | 4/510 [00:13<28:15,  3.35s/it]


  1%|          | 5/510 [00:16<27:55,  3.32s/it]


  1%|          | 6/510 [00:19<27:33,  3.28s/it]


  1%|▏         | 7/510 [00:22<27:10,  3.24s/it]


  2%|▏         | 8/510 [00:25<26:57,  3.22s/it]


  2%|▏         | 9/510 [00:28<26:44,  3.20s/it]


  2%|▏         | 10/510 [00:31<26:31,  3.18s/it]


  2%|▏         | 11/510 [00:34<26:27,  3.18s/it]


  2%|▏         | 12/510 [00:37<26:13,  3.16s/it]


  3%|▎         | 13/510 [00:40<26:06,  3.15s/it]


  3%|▎         | 14/510 [00:43<25:58,  3.14s/it]


  3%|▎         | 15/510 [00:47<25:52,  3.14s/it]


  3%|▎         | 16/510 [00:50<25:47,  3.13s/it]


  3%|▎         | 17/510 [00:53<25:39,  3.12s/it]


  4%|▎         | 18/510 [00:56<25:35,  3.12s/it]


  4%|▎         | 19/510 [00:59<25:28,  3.11s/i

 27%|██▋       | 136/510 [07:39<21:04,  3.38s/it]


 27%|██▋       | 137/510 [07:43<21:02,  3.39s/it]


 27%|██▋       | 138/510 [07:47<21:00,  3.39s/it]


 27%|██▋       | 139/510 [07:51<20:57,  3.39s/it]


 27%|██▋       | 140/510 [07:54<20:54,  3.39s/it]


 28%|██▊       | 141/510 [07:58<20:52,  3.39s/it]


 28%|██▊       | 142/510 [08:02<20:49,  3.39s/it]


 28%|██▊       | 143/510 [08:05<20:46,  3.40s/it]


 28%|██▊       | 144/510 [08:09<20:43,  3.40s/it]


 28%|██▊       | 145/510 [08:12<20:40,  3.40s/it]


 29%|██▊       | 146/510 [08:16<20:38,  3.40s/it]


 29%|██▉       | 147/510 [08:20<20:35,  3.40s/it]


 29%|██▉       | 148/510 [08:23<20:31,  3.40s/it]


 29%|██▉       | 149/510 [08:26<20:27,  3.40s/it]


 29%|██▉       | 150/510 [08:30<20:24,  3.40s/it]


 30%|██▉       | 151/510 [08:33<20:21,  3.40s/it]


 30%|██▉       | 152/510 [08:37<20:18,  3.40s/it]


 30%|███       | 153/510 [08:40<20:14,  3.40s/it]


 30%|███       | 154/510 [08:43<20:11,  3.40s/it]


 30%|███    

 53%|█████▎    | 270/510 [15:54<14:08,  3.54s/it]


 53%|█████▎    | 271/510 [15:58<14:05,  3.54s/it]


 53%|█████▎    | 272/510 [16:02<14:01,  3.54s/it]


 54%|█████▎    | 273/510 [16:05<13:57,  3.54s/it]


 54%|█████▎    | 274/510 [16:08<13:54,  3.53s/it]


 54%|█████▍    | 275/510 [16:12<13:50,  3.53s/it]


 54%|█████▍    | 276/510 [16:15<13:47,  3.54s/it]


 54%|█████▍    | 277/510 [16:19<13:43,  3.54s/it]


 55%|█████▍    | 278/510 [16:23<13:40,  3.54s/it]


 55%|█████▍    | 279/510 [16:27<13:37,  3.54s/it]


 55%|█████▍    | 280/510 [16:30<13:33,  3.54s/it]


 55%|█████▌    | 281/510 [16:34<13:30,  3.54s/it]


 55%|█████▌    | 282/510 [16:38<13:26,  3.54s/it]


 55%|█████▌    | 283/510 [16:42<13:23,  3.54s/it]


 56%|█████▌    | 284/510 [16:45<13:20,  3.54s/it]


 56%|█████▌    | 285/510 [16:48<13:16,  3.54s/it]


 56%|█████▌    | 286/510 [16:53<13:13,  3.54s/it]


 56%|█████▋    | 287/510 [16:56<13:10,  3.54s/it]


 56%|█████▋    | 288/510 [16:59<13:06,  3.54s/it]


 57%|█████▋ 

 79%|███████▉  | 404/510 [24:18<06:22,  3.61s/it]


 79%|███████▉  | 405/510 [24:21<06:18,  3.61s/it]


 80%|███████▉  | 406/510 [24:24<06:15,  3.61s/it]


 80%|███████▉  | 407/510 [24:28<06:11,  3.61s/it]


 80%|████████  | 408/510 [24:32<06:08,  3.61s/it]


 80%|████████  | 409/510 [24:35<06:04,  3.61s/it]


 80%|████████  | 410/510 [24:40<06:00,  3.61s/it]


 81%|████████  | 411/510 [24:44<05:57,  3.61s/it]


 81%|████████  | 412/510 [24:49<05:54,  3.62s/it]


 81%|████████  | 413/510 [24:52<05:50,  3.61s/it]


 81%|████████  | 414/510 [24:56<05:46,  3.61s/it]


 81%|████████▏ | 415/510 [24:59<05:43,  3.61s/it]


 82%|████████▏ | 416/510 [25:03<05:39,  3.61s/it]


 82%|████████▏ | 417/510 [25:06<05:35,  3.61s/it]


 82%|████████▏ | 418/510 [25:09<05:32,  3.61s/it]


 82%|████████▏ | 419/510 [25:13<05:28,  3.61s/it]


 82%|████████▏ | 420/510 [25:16<05:24,  3.61s/it]


 83%|████████▎ | 421/510 [25:19<05:21,  3.61s/it]


 83%|████████▎ | 422/510 [25:23<05:17,  3.61s/it]


 83%|███████

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_images/video1-2.mp4 

CPU times: user 25min 9s, sys: 5min 8s, total: 30min 18s
Wall time: 30min 39s


In [26]:
from moviepy.editor import *

clip = VideoFileClip("test_images/douyin3-3.mp4")
clip.write_gif("final.gif")


[MoviePy] Building file final.gif with imageio



100%|██████████| 196/196 [00:41<00:00,  4.73it/s]


In [19]:
# https://stackoverflow.com/questions/28361056/how-to-resize-a-video-clip-python
# resize the video
# BUT IT IS NOT WORKING
# clip = VideoFileClip("test_images/video1-1.mp4")
# clip_resized = clip.resize(width=1080, height=1920) # make the height 360px ( According to moviePy documenation The width is then computed so that the width/height ratio is conserved.)
# clip_resized.write_videofile("test_images/movie_1.mp4")

# 如果上面的转换不管用，就可以用如下命令
# https://stackoverflow.com/questions/28361056/how-to-resize-a-video-clip-python
# ffmpeg -i movie.mp4 -vf scale=750:1334 movie_360p.mp4

[MoviePy] >>>> Building video test_images/movie_1.mp4
[MoviePy] Writing audio in movie_1TEMP_MPY_wvf_snd.mp3



100%|██████████| 375/375 [00:00<00:00, 468.31it/s]

[MoviePy] Done.
[MoviePy] Writing video test_images/movie_1.mp4



 35%|███▌      | 179/510 [01:01<01:52,  2.93it/s]


 67%|██████▋   | 340/510 [02:05<01:02,  2.70it/s]


 97%|█████████▋| 496/510 [03:13<00:05,  2.56it/s]


100%|██████████| 510/510 [03:19<00:00,  2.56it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_images/movie_1.mp4 

